In [23]:
import matplotlib.pyplot as plt
import numpy as np
import os
import glob as gb
import cv2
import shutil
from tensorflow import keras
import tensorflow as tf
from keras.layers import Dense, Conv2D, MaxPooling2D, AvgPool2D, Input, Dropout, Flatten, BatchNormalization
from keras import layers
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
trainpath ='/content/drive/MyDrive/covid19/dataset-covid/train/'
testpath='/content/drive/MyDrive/covid19/dataset-covid/test/'
Valpath='/content/drive/MyDrive/covid19/dataset-covid/val/'

In [ ]:
#preprocessing data set
#convert image to RGB
#resize Image
#label image

In [26]:
#x_train,y_train
#due to session crash i will take only 500 image
new_size=224    
X_train = []
y_train = []
for folder in  os.listdir(trainpath) : 
    print( 'folder name is : ', folder)
    files = gb.glob(pathname= str( trainpath  + folder + '/*.png'))
    print( 'numbers of images in folder are : ', len(files))
    count=0
    for file in files: 
        image_class = {'covid': 0, 'Lung-Opacity': 1,'Normal':2,'viral-Pneumonia':3}
        orignal_image = cv2.imread(file)
        image = cv2.cvtColor(orignal_image, cv2.COLOR_BGR2RGB)
        resized_image = cv2.resize(image , (new_size,new_size))
        X_train.append(resized_image)
        y_train.append(image_class[folder])
        count=count+1
        if(count==300):
          break

print("X_train:       ",len(X_train) ) 
print("y_train:       ",len(y_train) ) 

folder name is :  covid
numbers of images in folder are :  2531
folder name is :  Lung-Opacity
numbers of images in folder are :  4208
folder name is :  Normal
numbers of images in folder are :  1508
folder name is :  viral-Pneumonia
numbers of images in folder are :  941
X_train:        1200
y_train:        1200


In [27]:
#300
#x_test,y_test
new_size=224    
X_test = []
y_test = []
for folder in  os.listdir(testpath) : 
    print( 'folder name is : ', folder)
    files = gb.glob(pathname= str( testpath  + folder + '/*.png'))
    print( 'numbers of images in folder are : ', len(files))
    count=0
    for file in files: 
        image_class = {'covid': 0, 'Lung-Opacity': 1,'Normal':2,'viral-Pneumonia':3}
        orignal_image = cv2.imread(file)
        image = cv2.cvtColor(orignal_image, cv2.COLOR_BGR2RGB)
        resized_image = cv2.resize(image , (new_size,new_size))
        X_test.append(resized_image)
        y_test.append(image_class[folder])
        count=count+1
        if(count==100):
          break

print("X_test:       ",len(X_test) ) 
print("y_test:       ",len(y_test) ) 

folder name is :  covid
numbers of images in folder are :  724
folder name is :  Lung-Opacity
numbers of images in folder are :  1203
folder name is :  Normal
numbers of images in folder are :  2039
folder name is :  viral-Pneumonia
numbers of images in folder are :  270
X_test:        400
y_test:        400


In [28]:
#x_validation ,y_validation
#200
new_size=224    
X_val = []
y_val = []
for folder in  os.listdir(Valpath ) : 
    print( 'folder name is : ', folder)
    files = gb.glob(pathname= str( Valpath  + folder + '/*.png'))
    print( 'numbers of images in folder are : ', len(files))
    count=0
    for file in files: 
        image_class = {'covid': 0, 'Lung-Opacity': 1,'Normal':2,'viral-Pneumonia':3}
        orignal_image = cv2.imread(file)
        image = cv2.cvtColor(orignal_image, cv2.COLOR_BGR2RGB)
        resized_image = cv2.resize(image , (new_size,new_size))
        X_val.append(list(resized_image))
        y_val.append(image_class[folder])
        count=count+1
        if(count==200):
          break
print("X_val is:       ",len(X_val) ) 
print("y_val is:       ",len(y_val))

folder name is :  covid
numbers of images in folder are :  361
folder name is :  Lung-Opacity
numbers of images in folder are :  601
folder name is :  Normal
numbers of images in folder are :  1019
folder name is :  viral-Pneumonia
numbers of images in folder are :  134
X_val is:        734
y_val is:        734


In [29]:
#convert to numpy array
X_train = np.array(X_train)
y_train = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [30]:
#using categorical method
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train,4)
y_val = to_categorical(y_val,4)
y_test = to_categorical(y_test,4)

In [31]:
#shuffel data
from sklearn.utils import shuffle
X_train,y_train = shuffle(X_train,y_train)
X_val,y_val = shuffle(X_val,y_val)
X_test,y_test = shuffle(X_test,y_test)

In [32]:
#last step using normalization
X_train=X_train/255.0
X_val=X_val/255.0
X_test=X_test/255.0

In [ ]:
#VIT transformer

In [33]:
!pip install -U tensorflow-addons

!pip install vit-keras
from vit_keras import vit



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 29.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
image_size = 224
model = vit.vit_l32(
    image_size=image_size,
    activation='softmax',
    pretrained=True,
    include_top=True,
    pretrained_top=False,
    classes=4
)

1226670080/1226658854 [==============================] - 261s 0us/step


/usr/local/lib/python3.7/dist-packages/vit_keras/utils.py:83: UserWarning: Resizing position embeddings from 12, 12 to 7, 7
  UserWarning,


In [35]:
model.summary()

Model: "vit-l32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 embedding (Conv2D)          (None, 7, 7, 1024)        3146752   
                                                                 
 reshape (Reshape)           (None, 49, 1024)          0         
                                                                 
 class_token (ClassToken)    (None, 50, 1024)          1024      
                                                                 
 Transformer/posembed_input   (None, 50, 1024)         51200     
 (AddPositionEmbs)                                               
                                                                 
 Transformer/encoderblock_0   ((None, 50, 1024),       12596224  
 (TransformerBlock)           (None, 16, None, None))      

In [36]:
#train last layer
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
epochs = 10
earlystop = EarlyStopping("val_accuracy", patience=5, verbose=1, restore_best_weights=True,)
history = model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=epochs,callbacks=[earlystop])

Epoch 1/10
38/38 [==============================] - 97s 2s/step - loss: 1.8145 - accuracy: 0.2733 - val_loss: 1.4426 - val_accuracy: 0.1826
Epoch 2/10
38/38 [==============================] - 49s 1s/step - loss: 1.4209 - accuracy: 0.2483 - val_loss: 1.3884 - val_accuracy: 0.2725
Epoch 3/10
38/38 [==============================] - 48s 1s/step - loss: 1.4284 - accuracy: 0.2325 - val_loss: 1.4025 - val_accuracy: 0.2725
Epoch 4/10
38/38 [==============================] - 48s 1s/step - loss: 1.4058 - accuracy: 0.2450 - val_loss: 1.4180 - val_accuracy: 0.1826
Epoch 5/10
38/38 [==============================] - 48s 1s/step - loss: 1.4033 - accuracy: 0.2475 - val_loss: 1.3749 - val_accuracy: 0.2725
Epoch 6/10
38/38 [==============================] - 48s 1s/step - loss: 1.4036 - accuracy: 0.2408 - val_loss: 1.3749 - val_accuracy: 0.2725
Epoch 7/10
38/38 [==============================] - 49s 1s/step - loss: 1.4090 - accuracy: 0.2575 - val_loss: 1.3789 - val_accuracy: 0.2725
Epoch 7: early stopp

In [37]:
model.evaluate(X_test,y_test)

13/13 [==============================] - 4s 335ms/step - loss: 1.4094 - accuracy: 0.2500


[1.409368634223938, 0.25]